In [1]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf

In [2]:
def fetch_stocks():
    # filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    filters = ['cap_largeover', 'sh_avgvol_o2000']
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [3]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.46it/s]


In [4]:
def cal_VRRS(ticker, ref, vol_weight, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * vol_weight * 100

In [5]:
ticker_dict = {}

In [6]:
# Get data from Yahoo Finance for the S&P 500 and the VIX index:    
spy = yf.Ticker("SPY")
spy = spy.history(period="max")['Close']  

for stock in stock_list:
    
    ticker = stock['Ticker']

    stock = yf.Ticker(ticker)    
    stock = stock.history(period="max")['Close']  

    # Calculate the VRRS indicator:
    rolling_length = 21
    vol_weight = 1
    
    vrrs = cal_VRRS(stock, spy, vol_weight, rolling_length)
    
    ticker_dict[ticker] = vrrs.iloc[-1]
    
    print(ticker, vrrs.iloc[-1])

GRAB -5.740986279744983
SAN -7.525045076731969
ABEV 2.373365286909414
BBD -7.656042701584467
NU 1.87411874759805
ITUB -5.98403107560508
BBVA -3.789369108273091
NOK -2.67490097203364
ASX -11.809928966351247
WIT -5.602461567195588
NWG -4.143980414286355
MUFG -2.7064563567058677
CS -6.44097725053739
LU -9.585305761398386
SMFG -0.4226389707855415
SIRI 1.905617087741301
UMC -8.90884613914215
BCS -6.170756769938504
ERIC -2.322956520675068
EBR -2.816362261308483
RKT 11.420153501602206
ING -7.232241866542423
ET -6.216772450271195
PLTR 10.811310661569316
PCG -1.8319508330883958
VTRS -5.4542431870743755
OWL -2.7364448963637935
LUMN -2.9686133479354018
PBR-A -1.7848431372155908
CNHI -9.023671135850272
F -3.4479194058044533
PBR -3.537309369911912
HBAN -1.744341425720251
AMCR -1.1864730382612443
STLA -4.52593196473855
HPE -5.073414632978823
AQN -1.6854065070934756
WBD -3.8743494313074667
TAK 3.0717451297746967
SNAP 8.013020224258714
VOD -2.332584672871066
CPNG 18.19317431067682
HST -5.9696043684727

AMGN 0.981790587447489
DG 3.103220427943135
MCD 1.8463169825287677
AMT -0.3029734529653343
DHR 2.712864923551262
MSFT 2.012506431156622
LIN -8.762823791105474
ACN -3.1373358509933023
BRK-B -0.9799014588159942
HD 0.3737942639275148
MA -1.9179899435815366
LLY 4.298228428518158
SPGI 3.798549867394278
ADBE 0.7583640301282322
AVGO -3.0524600645204325
COST 4.479043167409526
UNH 3.5473616551583738
TSLA 6.025927865067565


In [7]:
sorted_x = sorted(ticker_dict.items(), key=lambda x: x[1])

In [8]:
sorted_x

[('TECK', -22.258381396523106),
 ('FCX', -16.656708729803942),
 ('MRO', -14.22973397610705),
 ('MT', -13.38976273792531),
 ('OVV', -13.320038721701955),
 ('COP', -13.113855016921574),
 ('SLB', -12.869670213821326),
 ('HAL', -12.847507365469339),
 ('FANG', -12.841527886658774),
 ('PSX', -12.368028235633332),
 ('EOG', -12.315440148169495),
 ('DVN', -12.23858721809448),
 ('EQT', -12.04942660900393),
 ('CVE', -11.98581007410192),
 ('SU', -11.837245032642823),
 ('ASX', -11.809928966351247),
 ('GOLD', -11.061935854670194),
 ('MPC', -10.783301786896338),
 ('DELL', -10.699857511008256),
 ('BHP', -10.438887870717863),
 ('VLO', -10.43657334288036),
 ('HES', -10.283510540541021),
 ('PXD', -10.264138943860518),
 ('RIO', -10.212123656387044),
 ('MOS', -10.12989607501429),
 ('AEM', -9.593421120347708),
 ('LU', -9.585305761398386),
 ('CTRA', -9.466724113215884),
 ('WPM', -9.403741194790793),
 ('NTR', -9.104279641319616),
 ('CNHI', -9.023671135850272),
 ('CVX', -9.012865958989869),
 ('UMC', -8.9088461